In [ ]:
import sys
sys.path.append("../../")
sys.path.append("../../features")

from PreProcessing import PreProcessing
from ExtractFeatures import extract_features
from features.GaborFilter.GaborFilter import generate_gabor_kernel_cv 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage as ski
from sklearn.model_selection import train_test_split
import tqdm
import pandas as pd
import os
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score



Model Training Pipeline

In [ ]:
# 1. Preprocess the images
# 2. Extract features
# 3. Train the model
# 4. Test the model

def main():

    folder_name = ["IBM Plex Sans Arabic" ,"Lemonada" , "Marhey" , "Scheherazade New" ]

    for j in range(0 , 4 ) : 
        for i in tqdm(range(0, 1000)): 
            img = ski.io.imread("../../data/raw/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg")
            
            if len(np.unique(img)) == 1 : 
                continue # all image contain the same color 

            preprocessed_image = PreProcessing(img)
            ski.io.imsave("../../data/processed/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg", preprocessed_image)

    # 2. Extract features
    # Extracting features
    columns = ["area_ratio"]
    for j in range(1 , 17):
        columns.append("gabor_mean_" + str(j))
        columns.append("gabor_std_" + str(j))
        columns.append("gabor_energy_" + str(j))
    columns.append("font_type")
    features = pd.DataFrame(columns=columns)
    gabor_kernels = generate_gabor_kernel_cv()
    for j in range(0 , 4):
        for i in tqdm.tqdm(range(0, 1000)):
            pathname = "../../data/processed/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg"
            if os.path.exists(pathname):
                img = ski.io.imread(pathname)
                extractFeatures = extract_features(img,gabor_kernels)
                features = pd.concat([features , pd.DataFrame([np.concatenate([extractFeatures , [j]])] , columns = columns)])

    features.to_csv('./features_1000.csv')


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import warnings
warnings.filterwarnings("ignore")

dataset = pd.read_csv('../../features/features_1000_again.csv')    
# dataset2 = pd.read_csv('../../features/features_1000_wo_heggi.csv')
# dataset = pd.concat([dataset , dataset2]) 
# column split
x_data = dataset.drop(columns=['Unnamed: 0'])     
# Check the number of unique values in each column
unique_counts = dataset.nunique()

# Get the column names where the number of unique values is 1
columns_to_drop = unique_counts[unique_counts == 1].index

# Drop the columns
x_data = dataset.drop(columns=columns_to_drop)

# print(dataset.columns)

x_data = x_data.iloc[:, :-1]                                   
y_data = dataset['font_type']  
scaler = StandardScaler()

print(x_data.shape)
scaler.fit(x_data)
x_data = scaler.transform(x_data)
# split the data to avoid overfitting
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=42)

svm_model = svm.SVC(kernel='poly', C=10, gamma='auto', degree=3, coef0=1)
scores = cross_val_score(svm_model, x_train, y_train, cv=10)
print("Scores:", scores)
print("Mean:", scores.mean())

svm_model2 = svm.SVC(kernel='poly', C=10, gamma='auto', degree=3, coef0=1)
# svm_model2.fit(x_train, y_train)
svm_model2.fit(x_data, y_data)
# make pickle file
import pickle
filename = 'svm.pkl'
filename2 = 'scaler.pkl'
pickle.dump(svm_model2, open(filename, 'wb'))
pickle.dump(scaler, open(filename2, 'wb'))
y_pred = svm_model2.predict(x_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
# Create an instance of the QDA classifier
# qda = QuadraticDiscriminantAnalysis()
# scores = cross_val_score(qda, x_train, y_train, cv=5)
# print("qda _ Scores:", scores)
# print("qda_mean:", scores.mean())

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# load the pickle file
import pickle
import tqdm
filename = 'svm.pkl'
scaler_filename = 'scaler.pkl'
loaded_model = pickle.load(open(filename, 'rb'))
scaler = pickle.load(open(scaler_filename, 'rb'))
import pandas as pd
import numpy as np

feature_names=['Area Ratio']
for i in range(1, 17):
    feature_names.append('gabor mean ' + str(i))
    feature_names.append('gabor std ' + str(i))
    feature_names.append('gabor energy ' + str(i))

for i in range(1, 6):
    feature_names.append('glcm ' + str(i))
for i in range(1, 27):
    feature_names.append('lbp ' + str(i))



folder_name = ["IBM Plex Sans Arabic" ,"Lemonada" , "Marhey" , "Scheherazade New" ]
import skimage as ski
import numpy as np
kernels = []
thetas = [0 , np.pi/4 , np.pi/2 , 3*np.pi/4] # mentioned in the papers  0  , 45 , 90 , 135
for theta in thetas : 
    for frequency in [0.05, 0.01, 0.15, 0.3] : 
        kernel = ski.filters.gabor_kernel(frequency=frequency , theta = theta )
        kernels.append(np.real(kernel))


for i in tqdm.tqdm(range(0, 1000)): 
    for j in range(0 , 4 ) : 
        img = ski.io.imread("../../data/raw/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg")

        if len(np.unique(img)) == 1 : 
            continue # all image contain the same color 

        preprocessed_image = PreProcessing.PreProcessing(img)
        # compare preprocessed image with image in processed folder
        # img2 = ski.io.imread("../../data/processed/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg")
        # display images
        # ski.io.imshow(preprocessed_image)
        # ski.io.show()
        # ski.io.imshow(img2)
        # ski.io.show()
        features = extract_features(preprocessed_image, kernels)
        # print("Features: ", features)

        features = scaler.transform([features])
        print("features shape: ", features.shape)
        # Assuming 'features' is a numpy array and 'feature_names' is a list of feature names
        font_type = loaded_model.predict(features)
        print("Font ", str(j), ": ", font_type[0])

